In [45]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [46]:
df = pd.read_csv("articles.csv",encoding="unicode_escape")
df.head()

,author,claps,reading_time,link,title,text
0,Justin Lee,8.3K,11,https://medium.com/swlh/chatbots-were-the-next...,Chatbots were the next big thing: what happene...,"Oh, how the headlines blared:\nChatbots were T..."
1,Conor Dewey,1.4K,7,https://towardsdatascience.com/python-for-data...,Python for Data Science: 8 Concepts You May Ha...,If youâve ever found yourself looking up the...
2,William Koehrsen,2.8K,11,https://towardsdatascience.com/automated-featu...,Automated Feature Engineering in Python â To...,Machine learning is increasingly moving from h...
3,Gant Laborde,1.3K,7,https://medium.freecodecamp.org/machine-learni...,Machine Learning: how to go from Zero to Hero ...,If your understanding of A.I. and Machine Lear...
4,Emmanuel Ameisen,935,11,https://blog.insightdatascience.com/reinforcem...,Reinforcement Learning from scratch â Insigh...,Want to learn about applied Artificial Intelli...


In [47]:
df['text'][0]

"Oh, how the headlines blared:\nChatbots were The Next Big Thing.\nOur hopes were sky high. Bright-eyed and bushy-tailed, the industry was ripe for a new era of innovation: it was time to start socializing with machines.\nAnd why wouldnâ\x80\x99t they be? All the road signs pointed towards insane success.\nAt the Mobile World Congress 2017, chatbots were the main headliners. The conference organizers cited an â\x80\x98overwhelming acceptance at the event of the inevitable shift of focus for brands and corporates to chatbotsâ\x80\x99.\nIn fact, the only significant question around chatbots was who would monopolize the field, not whether chatbots would take off in the first place:\nOne year on, we have an answer to that question.\nNo.\nBecause there isnâ\x80\x99t even an ecosystem for a platform to dominate.\nChatbots werenâ\x80\x99t the first technological development to be talked up in grandiose terms and then slump spectacularly.\nThe age-old hype cycle unfolded in familiar fashion...

In [48]:
df = df[["text"]]
df.head()

,text
0,"Oh, how the headlines blared:\nChatbots were T..."
1,If youâve ever found yourself looking up the...
2,Machine learning is increasingly moving from h...
3,If your understanding of A.I. and Machine Lear...
4,Want to learn about applied Artificial Intelli...


In [49]:
df = df.head(50)

In [50]:
print("No.of articles : ",len(df['text'].values))

No.of articles :  50


In [51]:
data = df['text'].values

### Preprocessing Steps

1. Remove HTML tags </br>
2. Removing Punctuation </br>
3. Removing special numbers or digits </br>
4. Removing special characters </br>
5. Removing extra whitespaces </br>
6. Removing stopwords </br>

In [52]:
from string import punctuation
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [53]:
remov = list(stopwords.words('english')) + list(punctuation)
remov.remove('not')

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Split text into sentences
    sentences = text.split('\n')
    
    # Initialize list to store preprocessed sentences
    preprocessed_sentences = []
    
    # Define regex pattern to remove special characters and punctuation
    pattern = r'[^a-zA-Z\s]'
    
    # Remove special characters, punctuation, and stopwords, and tokenize sentences
    for sentence in sentences:
        # Remove special characters and punctuation
        sentence = re.sub(pattern, '', sentence)
        
        # Tokenize sentence
        tokens = word_tokenize(sentence)
        
        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        #tokens = [word for word in tokens if word not in stop_words]
        tokens = [word for word in tokens]
        
        # Add preprocessed sentence to list
        preprocessed_sentences.append(tokens)
    
    return preprocessed_sentences

In [54]:
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Split text into sentences
    sentences = text.split('\n')

    # Initialize list to store preprocessed sentences
    preprocessed_sentences = []

    # Define regex pattern to remove special characters and punctuation
    pattern = r'[^a-zA-Z\s]'

    # Remove special characters, punctuation, and stopwords, and tokenize sentences
    for sentence in sentences:
        # Remove special characters and punctuation
        sentence = re.sub(pattern, '', sentence)

        # Tokenize sentence
        tokens = word_tokenize(sentence)

        # Remove stopwords
        stop_words = set(stopwords.words('english'))
        tokens = [word for word in tokens if word not in stop_words]

        # Join tokens into a single string
        preprocessed_sentence = ' '.join(tokens)

        # Add preprocessed sentence to list
        preprocessed_sentences.append(preprocessed_sentence)

    return preprocessed_sentences

In [55]:
# Preprocess text column
corpus = []
for text in df['text']:
    preprocessed_sentences = preprocess_text(text)
    corpus.extend(preprocessed_sentences)

In [56]:
# Print preprocessed corpus
#print(corpus)

In [57]:
print("Number of Sentences in Corpus : ", len(corpus))

Number of Sentences in Corpus :  2063


In [58]:
corpus[0]

'oh headlines blared'

In [59]:
corpus[1]

'chatbots next big thing'

In [60]:
corpus[0:3]

['oh headlines blared',
 'chatbots next big thing',
 'hopes sky high brighteyed bushytailed industry ripe new era innovation time start socializing machines']

In [61]:
tokenizer = Tokenizer() #instantiating the tokenizer
tokenizer.fit_on_texts(corpus) #creates tokens for each words
total_words = len(tokenizer.word_index) + 1 #calculating total number of words in the initial sentence

In [62]:
total_words

8039

In [63]:
#tokenizer.word_index

In [64]:
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    print(token_list)
    break

[2552, 4428, 4429]


In [65]:
input_sequences = [] #training features (x) will be a list

for line in corpus:
    #converts each sentence as its tokenized equivalent
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        #generating n gram sequences
        n_gram_sequence = token_list[:i+1]
        #appending each n gram sequence to the list of our features (xs)
        input_sequences.append(n_gram_sequence)

In [66]:
input_sequences[0]

[2552, 4428]

In [67]:
input_sequences[1]

[2552, 4428, 4429]

In [68]:
input_sequences[2]

[904, 93]

In [69]:
#calculating the length of the longest sequence
max_sequence_len = max([len(x) for x in input_sequences])
#pre-pading each value of the input_sequence
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
#creating xs and their labels using numpy slicing
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
#creating one hot encoding values
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [70]:
#creating a sequential model
model = Sequential()
#adding an embedding layer with 64 as the embedding dimension
model.add(Embedding(total_words, 64, input_length=max_sequence_len-1))
#adding 20 LSTM units
model.add(Bidirectional(LSTM(20)))
#creating a dense layer with 54 output units (total_words) with softmax activation
model.add(Dense(total_words, activation='softmax'))

In [71]:
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy'])

In [72]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 221, 64)           514496    
                                                                 
 bidirectional_1 (Bidirecti  (None, 40)                13600     
 onal)                                                           
                                                                 
 dense_1 (Dense)             (None, 8039)              329599    
                                                                 
Total params: 857695 (3.27 MB)
Trainable params: 857695 (3.27 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [73]:
history = model.fit(xs, ys, epochs=200, verbose=1)

Epoch 1/200
1515/1515 [==============================] - 98s 63ms/step - loss: 8.1835 - accuracy: 0.0157
Epoch 2/200
1515/1515 [==============================] - 39s 26ms/step - loss: 7.8099 - accuracy: 0.0204
Epoch 3/200
1515/1515 [==============================] - 36s 24ms/step - loss: 7.6661 - accuracy: 0.0209
Epoch 4/200
1515/1515 [==============================] - 35s 23ms/step - loss: 7.5161 - accuracy: 0.0245
Epoch 5/200
1515/1515 [==============================] - 34s 22ms/step - loss: 7.3535 - accuracy: 0.0287
Epoch 6/200
1515/1515 [==============================] - 34s 22ms/step - loss: 7.1850 - accuracy: 0.0354
Epoch 7/200
1515/1515 [==============================] - 33s 22ms/step - loss: 7.0202 - accuracy: 0.0407
Epoch 8/200
1515/1515 [==============================] - 32s 21ms/step - loss: 6.8670 - accuracy: 0.0462
Epoch 9/200
1515/1515 [==============================] - 31s 21ms/step - loss: 6.7203 - accuracy: 0.0520
Epoch 10/200
1515/1515 [==============================]

#predicting the next word using an initial sentence
input_phrase = "the weather today is sunny"
next_words = 5

for _ in range(next_words):
    #converting our input_phrase to tokens and excluding the out of vcabulary words
    token_list = tokenizer.texts_to_sequences([input_phrase])[0]
    #padding the input_phrase
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    #predicting the token of the next word using our trained model
    predicted = model.predict(token_list, verbose=0)
    output_word = "" #initialising output word as blank at the beginning
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            #converting the token back to the corresponding word and
            #storing it in the output_word
            output_word = word
            break
    input_phrase += " " + output_word
print(input_phrase)

In [74]:
import random

# Preprocess text column
corpus = []
for text in df['text']:
    preprocessed_sentences = preprocess_text(text)
    corpus.extend(preprocessed_sentences)

# Randomly select a sentence from the corpus
selected_sentence = random.choice(corpus)
print("Selected sentence from the input data:")
print(selected_sentence)

# Set the input_phrase as the selected sentence
input_phrase = selected_sentence

# Define the number of sentences to generate
num_sentences = 3

# Define the maximum length of the generated sentences
max_sentence_length = 20

# List to store the generated sentences
generated_sentences = []

# Generate additional sentences of similar context
for _ in range(num_sentences):
    generated_sentence = input_phrase.split()  # Initialize with the input phrase

    # Generate the next word until reaching maximum length or end-of-sentence token
    for _ in range(max_sentence_length):
        # Tokenize the input_phrase and pad it
        token_list = tokenizer.texts_to_sequences([input_phrase])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        # Predict the next word using the trained model
        predicted = model.predict(token_list, verbose=0)

        # Sample the next word from the predicted probability distribution
        predicted_word_index = np.random.choice(len(predicted[0]), p=predicted[0])

        # Convert the token back to the corresponding word
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break

        # Append the predicted word to the generated sentence
        generated_sentence.append(output_word)

        # If the predicted word is an end-of-sentence token, stop generating
        if output_word == '.' or output_word == '?' or output_word == '!':
            break

        # Update the input_phrase with the new generated sentence
        input_phrase = ' '.join(generated_sentence)

    # Add the generated sentence to the list
    generated_sentences.append(' '.join(generated_sentence))

# Print the selected sentence and the final generated sentences
print("\nSelected sentence:")
print(selected_sentence)
print("\nGenerated sentences:")
for sentence in generated_sentences:
    print(sentence)


Selected sentence from the input data:
google translate invented language help translate effectively

Selected sentence:
google translate invented language help translate effectively

Generated sentences:
google translate invented language help translate effectively taking invented fully types profile even people enjoy web experience theyre right tech difficult lot hands providers however shes able
google translate invented language help translate effectively taking invented fully types profile even people enjoy web experience theyre right tech difficult lot hands providers however shes able learn let people better exactly evaluate let gives us know set information time table features ram modification resist referred method
google translate invented language help translate effectively taking invented fully types profile even people enjoy web experience theyre right tech difficult lot hands providers however shes able learn let people better exactly evaluate let gives us know set inform